### Tinker finetuning (SFT) — phishing classifier

This notebook takes `data/finetuning/training_data.json` and runs a **Tinker supervised fine-tune** (SFT).

It does 3 things:
1. Convert `training_data.json` → Tinker Cookbook conversation JSONL (`tinker_conversations.jsonl`).
2. Configure a supervised training run (base model + renderer + dataset builder + hyperparams).
3. Start training and show how to find the final `sampler_path` for inference.

**Prereqs**
- `TINKER_API_KEY` is set (we load `.env` if present).
- The sibling repo `tinker-cookbook` exists (typically at `../tinker-cookbook`).

**Safety**
This notebook is **safe-by-default**: it will refuse to overwrite any output files unless you set `OVERWRITE_OUTPUTS=True`.

**Outputs**
- `data/finetuning/tinker_conversations.jsonl` (or a versioned JSONL)
- Run logs under `LOG_PATH` (default: `/tmp/tinker-examples/sl_ar_phishing`)
- `sampler_path` (Tinker path) which you can use for inference (SamplingClient)


In [1]:
# Imports + environment

from __future__ import annotations

import json
import sys
from datetime import datetime
from pathlib import Path

from dotenv import load_dotenv

load_dotenv()
print("cwd:", Path.cwd())


cwd: /Users/ext-elias.melas/Documents/Gitcode/ar_finetune/notebooks


### 1) Make `tinker_cookbook` importable

We add the local `tinker-cookbook` repo to `sys.path` so we can import `tinker_cookbook` without pip-installing it.

If you already added it as a uv editable dependency (`uv add --editable <path_to>/tinker-cookbook`), you can skip this.


In [3]:
def find_tinker_cookbook_repo(start: Path) -> Path:
    candidates: list[Path] = []
    for p in [start, *start.parents]:
        candidates.append(p / "tinker-cookbook")
        candidates.append(p)

    for repo in candidates:
        if (repo / "tinker_cookbook" / "__init__.py").exists():
            return repo

    raise FileNotFoundError(
        "Could not find the tinker-cookbook repo (missing tinker_cookbook/__init__.py)."
    )

TINKER_COOKBOOK_REPO = find_tinker_cookbook_repo(Path.cwd())
if str(TINKER_COOKBOOK_REPO) not in sys.path:
    sys.path.insert(0, str(TINKER_COOKBOOK_REPO))

import tinker_cookbook  # noqa: F401
print("tinker-cookbook repo:", TINKER_COOKBOOK_REPO)
print("tinker_cookbook import ok")


tinker-cookbook repo: /Users/ext-elias.melas/Documents/Gitcode/tinker-cookbook
tinker_cookbook import ok


### 2) Convert `training_data.json` to Tinker conversation JSONL

This writes `data/finetuning/tinker_conversations.jsonl` **only if it doesn't already exist** (unless `OVERWRITE_OUTPUTS=True`).x

The training_data.json is in the form:
``
[
    {'instruction':'',
    'input':'',
    'output':''}
    ]
``

In [ ]:
# Safety switches (safe-by-default)
OVERWRITE_OUTPUTS = True
RUN_TAG = 'test'

def find_ar_finetune_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / "data/finetuning/training_data_spam.json").exists():
            return p
    raise FileNotFoundError("Could not find data/finetuning/training_data.json by searching upward from cwd.")

AR_FINETUNE_ROOT = find_ar_finetune_root(Path.cwd())
print("ar_finetune root:", AR_FINETUNE_ROOT)

in_path = AR_FINETUNE_ROOT / "data/finetuning/training_data_spam.json"
out_path = AR_FINETUNE_ROOT / "data/finetuning/tinker_conversations_spam.jsonl"

# Guard against accidental overwrite
if out_path.exists() and not OVERWRITE_OUTPUTS:
    raise FileExistsError(
        f"Refusing to overwrite existing file: {out_path}\n"
        f"Either set OVERWRITE_OUTPUTS=True, or change out_path (e.g. include a tag like {RUN_TAG})."
    )

data = json.loads(in_path.read_text(encoding="utf-8"))
assert isinstance(data, list)

out_path.parent.mkdir(parents=True, exist_ok=True)
with out_path.open("w", encoding="utf-8") as f:
    for ex in data:
        instruction = (ex.get("instruction") or "").strip()
        user_input = (ex.get("input") or "").strip()
        output = (ex.get("output") or "").strip()

        messages = [
            {"role": "system", "content": instruction},
            {"role": "user", "content": user_input},
            {"role": "assistant", "content": output},
        ]
        f.write(json.dumps({"messages": messages}, ensure_ascii=False) + "\n")

print("Wrote:", out_path)
print("Head:")
print(out_path.open("r", encoding="utf-8").read().splitlines()[0][:500])


ar_finetune root: /Users/ext-elias.melas/Documents/Gitcode/ar_finetune
Wrote: /Users/ext-elias.melas/Documents/Gitcode/ar_finetune/data/finetuning/tinker_conversations_spam.jsonl
Head:
{"messages": [{"role": "system", "content": "You are an email security analyst in a mobile gaming company. Your task is to classify if an email is spam or not.\n\nIf the email is spam, return \"SPAM\" for category, the reason it is spam, the reason keyword and the source.\nIf the email is not spam, return \"GENUINE\" for category, an empty string for reason, an empty string for reason_keyword and an empty list for source.\nIf unclear, return \"ASSIGN_TO_AGENT\" for category."}, {"role": "user", 


### 3) Configure the supervised training run

This matches the original notebook configuration.


In [8]:
import tinker
from tinker_cookbook import model_info
from tinker_cookbook.renderers import TrainOnWhat
from tinker_cookbook.supervised import train
from tinker_cookbook.supervised.data import FromConversationFileBuilder
from tinker_cookbook.supervised.types import ChatDatasetBuilderCommonConfig

MODEL_NAME = "meta-llama/Llama-3.1-8B"
RENDERER_NAME = model_info.get_recommended_renderer_name(MODEL_NAME)

common_config = ChatDatasetBuilderCommonConfig(
    model_name_for_tokenizer=MODEL_NAME,
    renderer_name=RENDERER_NAME,
    max_length=8192,
    batch_size=16,
    train_on_what=TrainOnWhat.ALL_ASSISTANT_MESSAGES,
)

dataset_builder = FromConversationFileBuilder(
    common_config=common_config,
    file_path=str(out_path.resolve()),
    test_size=128,
    shuffle_seed=0,
)

LOG_PATH = Path("/tmp/tinker-examples/sl_ar_phishing_spam").expanduser()

config = train.Config(
    log_path=str(LOG_PATH),
    model_name=MODEL_NAME,
    dataset_builder=dataset_builder,
    learning_rate=2e-4,
    lr_schedule="linear",
    num_epochs=1,
    eval_every=10,
    save_every=20,
)

print("Renderer:", RENDERER_NAME)
print("Dataset:", out_path.resolve())
print("Log path:", config.log_path)


/Users/ext-elias.melas/Documents/Gitcode/ar_finetune/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Renderer: role_colon
Dataset: /Users/ext-elias.melas/Documents/Gitcode/ar_finetune/data/finetuning/tinker_conversations_spam.jsonl
Log path: /tmp/tinker-examples/sl_ar_phishing_spam


### 4) Start training

Run:

```python
await train.main(config)
```


In [9]:
# Start training
await train.main(config)


tinker_cookbook.utils.ml_log:475 [INFO] Logging to: /tmp/tinker-examples/sl_ar_phishing_spam
tinker.lib.public_interfaces.service_client:66 [INFO] ServiceClient initialized for session ab71f6d1-f9b1-5f68-b387-59bb7b34c066
tinker.lib.public_interfaces.service_client:147 [INFO] TrainingClient initialized for model ab71f6d1-f9b1-5f68-b387-59bb7b34c066:train:0
tinker_cookbook.supervised.train:229 [INFO] Training for 271 batches x 1 epochs = 271 steps
tinker_cookbook.supervised.train:344 [INFO] Starting epoch 0
tinker_cookbook.utils.misc_utils:20 [INFO] Starting get_batch
tinker_cookbook.utils.misc_utils:23 [INFO] get_batch took 0.01 seconds
tinker_cookbook.supervised.train:259 [INFO] <|begin_of_text|>System: You are an email security analyst in a mobile gaming company. Your task is to classify if an email is spam or not.

If the email is spam, return "SPAM" for category, the reason it is spam, the reason keyword and the source.
If the email is not spam, return "GENUINE" for category, an em

tinker_cookbook.utils.ml_log:195 [INFO] 
                    Step 0                     
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Metric                         ┃ Value      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ epoch                          │ 0          │
│ learning_rate                  │ 0.000200   │
│ num_loss_tokens                │ 144.000000 │
│ num_sequences                  │ 16         │
│ num_tokens                     │ 7479       │
│ progress                       │ 0.003690   │
│ test/nll                       │ 2.981298   │
│ time/evals                     │ 3.801062   │
│ time/get_batch                 │ 0.015089   │
│ time/step                      │ 1.703909   │
│ time/total                     │ 5.538492   │
│ train_mean_nll                 │ 2.954231   │
└────────────────────────────────┴────────────┘
tinker_cookbook.utils.misc_utils:20 [INFO] Starting get_batch
tinker_cookbook.utils.misc_utils:23 [INFO] get_batch took 0.01 seconds
tinker_coo

tinker_cookbook.utils.ml_log:195 [INFO] 
                    Step 1                     
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Metric                         ┃ Value      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ epoch                          │ 0          │
│ learning_rate                  │ 0.000199   │
│ num_loss_tokens                │ 144.000000 │
│ num_sequences                  │ 16         │
│ num_tokens                     │ 5786       │
│ progress                       │ 0.007380   │
│ time/get_batch                 │ 0.011547   │
│ time/step                      │ 1.044229   │
│ time/total                     │ 2.786715   │
│ train_mean_nll                 │ 1.501719   │
└────────────────────────────────┴────────────┘
tinker_cookbook.utils.misc_utils:20 [INFO] Starting get_batch
tinker_cookbook.utils.misc_utils:23 [INFO] get_batch took 0.02 seconds
tinker_cookbook.supervised.train:259 [INFO] <|begin_of_text|>System: You are an email security analyst in 

tinker_cookbook.utils.ml_log:195 [INFO] 
                    Step 2                     
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Metric                         ┃ Value      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ epoch                          │ 0          │
│ learning_rate                  │ 0.000199   │
│ num_loss_tokens                │ 144.000000 │
│ num_sequences                  │ 16         │
│ num_tokens                     │ 8534       │
│ progress                       │ 0.011070   │
│ time/get_batch                 │ 0.014861   │
│ time/step                      │ 0.799910   │
│ time/total                     │ 1.891209   │
│ train_mean_nll                 │ 0.771756   │
└────────────────────────────────┴────────────┘
tinker_cookbook.utils.misc_utils:20 [INFO] Starting get_batch
tinker_cookbook.utils.misc_utils:23 [INFO] get_batch took 0.02 seconds
tinker_cookbook.supervised.train:259 [INFO] <|begin_of_text|>System: You are an email security analyst in 

tinker_cookbook.utils.ml_log:195 [INFO] 
                    Step 3                     
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Metric                         ┃ Value      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ epoch                          │ 0          │
│ learning_rate                  │ 0.000198   │
│ num_loss_tokens                │ 144.000000 │
│ num_sequences                  │ 16         │
│ num_tokens                     │ 13896      │
│ progress                       │ 0.014760   │
│ time/get_batch                 │ 0.020576   │
│ time/step                      │ 1.722341   │
│ time/total                     │ 2.573192   │
│ train_mean_nll                 │ 0.214607   │
└────────────────────────────────┴────────────┘
tinker_cookbook.utils.misc_utils:20 [INFO] Starting get_batch
tinker_cookbook.utils.misc_utils:23 [INFO] get_batch took 0.01 seconds
tinker_cookbook.supervised.train:259 [INFO] <|begin_of_text|>System: You are an email security analyst in 

...

tinker_cookbook.utils.ml_log:195 [INFO] 
                   Step 266                    
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Metric                         ┃ Value      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ epoch                          │ 0          │
│ learning_rate                  │ 0.000004   │
│ num_loss_tokens                │ 144.000000 │
│ num_sequences                  │ 16         │
│ num_tokens                     │ 12942      │
│ progress                       │ 0.985240   │
│ time/get_batch                 │ 0.020456   │
│ time/step                      │ 1.530078   │
│ time/total                     │ 2.204793   │
│ train_mean_nll                 │ 0.000431   │
└────────────────────────────────┴────────────┘
tinker_cookbook.utils.misc_utils:20 [INFO] Starting get_batch
tinker_cookbook.utils.misc_utils:23 [INFO] get_batch took 0.01 seconds
tinker_cookbook.supervised.train:259 [INFO] <|begin_of_text|>System: You are an email security analyst in 

tinker_cookbook.utils.ml_log:195 [INFO] 
                   Step 267                    
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Metric                         ┃ Value      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ epoch                          │ 0          │
│ learning_rate                  │ 0.000003   │
│ num_loss_tokens                │ 144.000000 │
│ num_sequences                  │ 16         │
│ num_tokens                     │ 6463       │
│ progress                       │ 0.988930   │
│ time/get_batch                 │ 0.014031   │
│ time/step                      │ 0.793572   │
│ time/total                     │ 2.357366   │
│ train_mean_nll                 │ 0.003592   │
└────────────────────────────────┴────────────┘
tinker_cookbook.utils.misc_utils:20 [INFO] Starting get_batch
tinker_cookbook.utils.misc_utils:23 [INFO] get_batch took 0.01 seconds
tinker_cookbook.supervised.train:259 [INFO] <|begin_of_text|>System: You are an email security analyst in 

tinker_cookbook.utils.ml_log:195 [INFO] 
                   Step 268                    
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Metric                         ┃ Value      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ epoch                          │ 0          │
│ learning_rate                  │ 0.000002   │
│ num_loss_tokens                │ 144.000000 │
│ num_sequences                  │ 16         │
│ num_tokens                     │ 4810       │
│ progress                       │ 0.992620   │
│ time/get_batch                 │ 0.009455   │
│ time/step                      │ 0.771067   │
│ time/total                     │ 1.599644   │
│ train_mean_nll                 │ 0.000170   │
└────────────────────────────────┴────────────┘
tinker_cookbook.utils.misc_utils:20 [INFO] Starting get_batch
tinker_cookbook.utils.misc_utils:23 [INFO] get_batch took 0.01 seconds
tinker_cookbook.supervised.train:259 [INFO] <|begin_of_text|>System: You are an email security analyst in 

tinker_cookbook.utils.ml_log:195 [INFO] 
                   Step 269                    
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Metric                         ┃ Value      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ epoch                          │ 0          │
│ learning_rate                  │ 0.000001   │
│ num_loss_tokens                │ 144.000000 │
│ num_sequences                  │ 16         │
│ num_tokens                     │ 8184       │
│ progress                       │ 0.996310   │
│ time/get_batch                 │ 0.014524   │
│ time/step                      │ 0.001223   │
│ time/total                     │ 3.456389   │
│ train_mean_nll                 │ 0.003226   │
└────────────────────────────────┴────────────┘
tinker_cookbook.utils.misc_utils:20 [INFO] Starting step
tinker_cookbook.utils.misc_utils:23 [INFO] step took 1.65 seconds
tinker_cookbook.utils.ml_log:143 [INFO] Wrote metrics to /tmp/tinker-examples/sl_ar_phishing_spam/metrics.jsonl


tinker_cookbook.utils.ml_log:195 [INFO] 
                   Step 270                    
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Metric                         ┃ Value      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ epoch                          │ 0          │
│ learning_rate                  │ 0.000001   │
│ num_loss_tokens                │ 144.000000 │
│ num_sequences                  │ 16         │
│ num_tokens                     │ 8950       │
│ progress                       │ 1.000000   │
│ test/nll                       │ 0.000598   │
│ time/evals                     │ 2.640547   │
│ time/get_batch                 │ 0.015080   │
│ time/step                      │ 1.649484   │
│ time/total                     │ 4.314168   │
│ train_mean_nll                 │ 0.000329   │
└────────────────────────────────┴────────────┘
tinker_cookbook.checkpoint_utils:81 [INFO] Saved checkpoints: {'state_path': 'tinker://ab71f6d1-f9b1-5f68-b387-59bb7b34c066:train:0/weights/fin

### 5) Get the final `sampler_path` (for inference)

After training finishes, use `checkpoints.jsonl` to grab the last `sampler_path`.


In [10]:
service_client = tinker.ServiceClient()

from tinker_cookbook.checkpoint_utils import get_last_checkpoint

ckpt_state = get_last_checkpoint(config.log_path, required_key="state_path")
ckpt_sampler = get_last_checkpoint(config.log_path, required_key="sampler_path")

print("Last state checkpoint:", ckpt_state.get("state_path") if ckpt_state else None)
print("Last sampler checkpoint:", ckpt_sampler.get("sampler_path") if ckpt_sampler else None)

if ckpt_sampler:
    sampling_client = service_client.create_sampling_client(model_path=ckpt_sampler["sampler_path"])
    print("Sampling client ready")
else:
    print("No sampler checkpoint found yet — make sure training finished and wrote checkpoints.jsonl.")


tinker.lib.public_interfaces.service_client:66 [INFO] ServiceClient initialized for session 52bd03d7-d96e-5289-b425-bab7b364d23f
tinker_cookbook.checkpoint_utils:24 [INFO] Reading checkpoints from /tmp/tinker-examples/sl_ar_phishing_spam/checkpoints.jsonl
tinker_cookbook.checkpoint_utils:46 [INFO] Found 14 valid checkpoints with key 'state_path' in /tmp/tinker-examples/sl_ar_phishing_spam
tinker_cookbook.checkpoint_utils:49 [INFO] Using last checkpoint: {'name': 'final', 'epoch': 1, 'batch': 271, 'state_path': 'tinker://ab71f6d1-f9b1-5f68-b387-59bb7b34c066:train:0/weights/final', 'sampler_path': 'tinker://ab71f6d1-f9b1-5f68-b387-59bb7b34c066:train:0/sampler_weights/final'}
tinker_cookbook.checkpoint_utils:24 [INFO] Reading checkpoints from /tmp/tinker-examples/sl_ar_phishing_spam/checkpoints.jsonl
tinker_cookbook.checkpoint_utils:46 [INFO] Found 14 valid checkpoints with key 'sampler_path' in /tmp/tinker-examples/sl_ar_phishing_spam
tinker_cookbook.checkpoint_utils:49 [INFO] Using last

Last state checkpoint: tinker://ab71f6d1-f9b1-5f68-b387-59bb7b34c066:train:0/weights/final
Last sampler checkpoint: tinker://ab71f6d1-f9b1-5f68-b387-59bb7b34c066:train:0/sampler_weights/final
Sampling client ready


### 6) (Optional) Download the LoRA adapter archive

Safe-by-default: downloads to a versioned filename and refuses to overwrite unless `OVERWRITE_OUTPUTS=True`.


In [12]:
import urllib.request

if ckpt_sampler is None:
    raise RuntimeError("No sampler checkpoint available to download.")

sampler_path = ckpt_sampler["sampler_path"]
rest_client = service_client.create_rest_client()
url_resp = rest_client.get_checkpoint_archive_url_from_tinker_path(sampler_path).result()

local_tar = Path(f"./tinker_adapter_archive_{RUN_TAG}.tar").resolve()
if local_tar.exists() and not OVERWRITE_OUTPUTS:
    raise FileExistsError(
        f"Refusing to overwrite existing file: {local_tar}. Set OVERWRITE_OUTPUTS=True to overwrite."
    )

urllib.request.urlretrieve(url_resp.url, str(local_tar))
print("Downloaded to:", local_tar)


Downloaded to: /Users/ext-elias.melas/Documents/Gitcode/ar_finetune/notebooks/tinker_adapter_archive_spam.tar
